In [1]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib as plt
import re

import nltk
from nltk.corpus import stopwords 
from html.parser import HTMLParser

In [2]:
prof = pd.read_csv("../professionals.csv")
answ = pd.read_csv("../answers.csv")
stud = pd.read_csv("../students.csv")
ques = pd.read_csv("../questions.csv")
emai = pd.read_csv("../emails.csv")
matc = pd.read_csv("../matches.csv")
tagq = pd.read_csv("../tag_questions.csv")
taqu = pd.read_csv("../tag_users.csv")